In [1]:
# Build a customer support assistant. Make the assistant to be able to:
    # Use tools to respond to questions
    # Connect with a human if needed
    # Be able to pause the process indefinitely and resume whenever the human responds

import json

from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults

from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import END, MessageGraph
from langgraph.prebuilt.tool_node import ToolNode


# Define the function that determines whether to continue or not
def should_continue(messages):
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return END
    else:
        return "action"


# Define a new graph
workflow = MessageGraph()

tools = [TavilySearchResults(max_results=1)]
model = ChatAnthropic(model="claude-3-haiku-20240307").bind_tools(tools)
workflow.add_node("agent", model)
workflow.add_node("action", ToolNode(tools))

workflow.set_entry_point("agent")

# Conditional agent -> action OR agent -> END
workflow.add_conditional_edges(
    "agent",
    should_continue,
)

# Always transition `action` -> `agent`
workflow.add_edge("action", "agent")

memory = SqliteSaver.from_conn_string(":memory:") # Here we only save in-memory

# Setting the interrupt means that any time an action is called, the machine will stop
app = workflow.compile(checkpointer=memory, interrupt_before=["action"])

In [2]:
# Run the graph
thread = {"configurable": {"thread_id": "4"}}
for event in app.stream("what is the weather in sf currently", thread, stream_mode="values"):
    event[-1].pretty_print()

================================ Human Message =================================

what is the weather in sf currently
================================== Ai Message ==================================

[{'id': 'toolu_0162yxpdqkahmyw1x3bRAK2Z', 'input': {'query': 'san francisco current weather'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_0162yxpdqkahmyw1x3bRAK2Z)
 Call ID: toolu_0162yxpdqkahmyw1x3bRAK2Z
  Args:
    query: san francisco current weather


In [4]:
for event in app.stream(None, thread, stream_mode="values"):
    event[-1].pretty_print()

================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1718761108, 'localtime': '2024-06-18 18:38'}, 'current': {'last_updated_epoch': 1718760600, 'last_updated': '2024-06-18 18:30', 'temp_c': 18.9, 'temp_f': 66.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 13.6, 'wind_kph': 22.0, 'wind_degree': 280, 'wind_dir': 'W', 'pressure_mb': 1010.0, 'pressure_in': 29.82, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 52, 'cloud': 75, 'feelslike_c': 18.9, 'feelslike_f': 66.0, 'windchill_c': 12.9, 'windchill_f': 55.2, 'heatindex_c': 14.2, 'heatindex_f': 57.6, 'dewpoint_c': 7.8, 'dewpoint_f': 46.1, 'vis_km': 16.0, '